In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from tqdm import tqdm
import random
import uuid

# Creación de datos sintéticos

El conjunto de datos creado contiene textos generados por GPT2 para cada una de las 6 categorías, lo que significa que cada ejemplo tiene una etiqueta asociada. Este es un conjunto de datos etiquetado que es esencial para entrenar un modelo de clasificación supervisada.

Este enfoque es útil cuando no se tiene acceso a un gran conjunto de datos etiquetados. GPT-2 genera ejemplos de texto que simulan lo que un humano podría escribir para cada categoría, lo que proporciona un volumen significativo de datos sintéticos que pueden ser utilizados para entrenar el modelo.

In [2]:
# Verificar si hay una GPU disponible
device_str = "cuda" if torch.cuda.is_available() else "cpu"  # "cuda" para GPU, "cpu" para CPU
print(device_str)

cpu


In [3]:
# Cargar el modelo y el tokenizador de GPT-2
model_name = "gpt2"  
model = GPT2LMHeadModel.from_pretrained(model_name).to(device_str)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

En el proceso de generación de datos sintéticos utilizando GPT-2 para entrenar un modelo de clasificación de texto, se probaron diferentes prompts (frases iniciales) para generar texto de manera que pudiera ser etiquetado correctamente en las seis categorías definidas: urgente, inteligencia artificial, computadoras, viajes, animales y ficción. Después de experimentar con varios prompts, se seleccionaron aquellos que generaban las salidas más coherentes y relevantes, asegurando que el texto generado correspondiera apropiadamente a cada categoría.

In [4]:
# Definir las categorías y la cantidad de sentencias por categoría
candidate_labels = ["urgent", "artificial_intelligence", "computer", "travel", "animal", "fiction"]
candidate_prompts = {
    'urgent': 'Write a sentence that seems like an urgent request.',
    'artificial_intelligence': 'Write a sentence that can be labeled as class artificial intelligence.',
    'computer': 'Write a sentence that can be labeled as class computers.',
    'travel': 'Write a sentence that can be labeled as class travels.',
    'animal': 'Write a sentence that can be labeled as class animals.',
    'fiction': 'Write a sentence that can be labeled as class fiction.'
}


In [5]:

num_sentences = 4000  

# Función para generar texto usando GPT-2 con parámetros de variabilidad
def generate_text(prompt, max_length=50, temperature=0.9, top_p=0.95, top_k=60):
    
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device_str)

    # Generar texto utilizando el modelo
    outputs = model.generate(
        input_ids=inputs['input_ids'],  # Extraer input_ids del texto tokenizado
        attention_mask=inputs['attention_mask'],  # Extraer attention_mask
        max_length=max_length,
        no_repeat_ngram_size=2,  # Evitar n-gramas repetidos
        top_p=top_p,  # Muestreo por núcleo
        top_k=top_k,  # Muestreo por top-k
        temperature=temperature,  # Ajustar temperatura para variabilidad
        pad_token_id=tokenizer.eos_token_id,  # ID del token de padding
        do_sample=True  # Habilitar muestreo (para aleatoriedad)
    )

    # Decodificar el texto generado y devolver el resultado, omitiendo tokens especiales
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text[len(prompt):].strip()  # Devolver el texto excluyendo el prompt original

In [6]:
# Generar ejemplos para cada categoría
def create_data(label):
    # Crear una lista para almacenar los resultados
    labeled_data = []

    # Generar ejemplos para cada categoría
    print(f"Generating examples for category: {label}")
    for _ in tqdm(range(num_sentences), desc=f"Generating {label}", ncols=100):
        prompt = candidate_prompts[label]
        
        # un while en caso de que genere resultados menores a 30 palabras
        len_flag = True
        while len_flag:
            generated_text = generate_text(prompt)
            if len(generated_text.split()) > 30:
              len_flag = False
        # Agregar el texto y la categoría al resultado
        result_dict = {"text": generated_text, "label": label}
        labeled_data.append(result_dict)

    return labeled_data


In [7]:
# cantidad de iteraciones
iterations = 1

# Crear una lista vacía para almacenar los DataFrames
all_data = []

# Iterar a través de las iteraciones y etiquetas
for i in range(iterations):
    for label in candidate_labels:
        # Crear los datos etiquetados
        labeled_data = create_data(label)
        df_labeled = pd.DataFrame(labeled_data)
        all_data.append(df_labeled)

# Concatenar todos los DataFrames en uno solo
df_all = pd.concat(all_data, ignore_index=True)

# Guardar el DataFrame combinado como un solo archivo Parquet
output_path = f'gpt2_database_4k.parquet'
df_all.to_parquet(output_path, index=False)

print(f"Datos guardados en: {output_path}")

Generating examples for category: urgent


Generating urgent: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00,  1.39it/s]


Generating examples for category: artificial_intelligence


Generating artificial_intelligence: 100%|█████████████████████████████| 1/1 [00:00<00:00,  1.16it/s]


Generating examples for category: computer


...

Generating examples for category: animal


Generating animal: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00,  1.43it/s]


Generating examples for category: fiction


Generating fiction: 100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.50it/s]

Datos guardados en: test.parquet
